# Machine Learning - Practical 4

Names: {YOUR NAMES}

In [1]:
import numpy as np
import cvxopt
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.datasets import load_wine
from sklearn.tree import DecisionTreeClassifier

## Task 0: The Data

We will work with the data from Practical 3. Load the data and split it into a training and test set. You can re-use the data splitting function from Practical 2.

In [2]:
def split_data(X, y, frac=0.3, seed=1):
    if seed is not None:
        np.random.seed(seed)

    # ---------------- INSERT CODE ----------------------
    idx = X.shape[0]

    n_trainset = int(idx*(1-frac))  # size of the training set
    n_testset = idx-n_trainset  # size of the test set

    idx_shuffled = np.random.permutation(idx)

    test_idx = idx_shuffled[:n_testset]
    train_idx = idx_shuffled[n_testset:n_testset+n_trainset]

    X_test = X[test_idx]
    y_test = y[test_idx]

    X_train = X[train_idx]
    y_train = y[train_idx]
    

    # ---------------- END CODE -------------------------
    
    return X_train, X_test, y_train, y_test

In [3]:
# load data
X_2d, t_2d = np.load('data/nonlin_2d_data.npy')[:,:2], np.load('data/nonlin_2d_data.npy')[:, 2]

In [4]:
# split data
X_train, X_test, t_train, t_test = split_data(X_2d, t_2d, seed=1)

## Task 1: Support Vector Machines

First, you will implement a training algorithm for the Support Vector Machine (SVM). For solving the quadratic program, we provide a simple interface to the cvxopt library below.

In SVMs, each data sample $x_n$ has a corresponding lagrange multiplier $\alpha_n$ which indicates if $x_n$ is a support vector. In the latter case $\alpha_n > 0$ holds. 
The goal of learning the SVM is to figure out which samples are support vectors by learning $\mathbf{\alpha}$. The dual SVM optimizes the following quadratic program.

$$ \min \frac{1}{2} \sum_{n=1}^N \sum_{m=1}^N \alpha_n \alpha_m t_n t_m k(\mathbf{x}_n, \mathbf{x}_m) - \sum_{n=1}^N \alpha_n$$
subject to 
$$ 0 \leq \alpha_n \leq C $$
$$ \sum_{n=1}^N \alpha_n t_n = 0 $$ 

The quadratic program solver expects the following form:
$$ \min \frac{1}{2} \alpha^T P \alpha + \mathbf q^T \mathbf \alpha $$
subject to 

$$A \alpha = b$$
$$G \alpha \leq h $$

Here, $A$ and $G$ are matrices with one row per individual constraint. Similarly, $b$ (the intercept or bias) and $h$ are vectors with one element per individual constraint.

Having trained the SVM, a prediction for an input $\mathbf{x}$ is made by:

$$ y = sign([\sum_n^{N} \alpha_n t_n k(\mathbf{x}, \mathbf{x}_n)] + b)  $$


### Task 1.1
 
Use the code provided below as a basis to express the constrained optimization problem in terms of $P, q, A, b, G$ and $h$ and implement a function `fit_svm` which passes these variables to the provided QP solver. Fit a SVM on the training data and extract its parameters using a linear kernel (dot product).

**Hints:**
  - The box constraint $0 \leq \alpha_n \leq C$ defines two constraints of the form $G \alpha_n \leq h$ for each $\alpha_n$.
  - The inequality $x \geq 0$ is equivalent to $-x \leq 0$.
  - The SVM is described in chapter 12 of Elements of Statistical Learning.

In [13]:
def linear_kernel(a, b):
    return a@b
    # ---------------- INSERT CODE ----------------------



    # ---------------- END CODE -------------------------


def fit_svm(X, t, kernel, C=1.0):
        #'''Fit SVM using data (X,t), specified kernel and parameter C.
    #Inputs
        #X:  predictors
        #t:  targets
        #C:  constant
    #'''
    t = np.array([-1. if l == 0 else 1. for l in t])
    N=len(t)
    # ---------------- INSERT CODE ----------------------
    P =np.identity(N)
    for i in range(N):
        for j in range(N):
            P[i][j]=t[i]*t[j]*kernel(X[i],X[j])
    q = -1*np.ones(N)
    A = t.reshape(1,N)
    print(np.shape(A))
    b = np.zeros(1).reshape(1,1)
    G = np.zeros(2*N*N)
    G=G.reshape(2*N,N)
    for i in range(N):
        G[i,i]=1
        G[N+i,i]
    h=np.array([C for i in range(N)])
    h=np.append(h,np.zeros(N))     
    # ---------------- END CODE -------------------------
    n_samples=N
    assert P.shape == (len(X), len(X))
    assert len(q) == len(X)
    assert A.shape == (1, n_samples) 
    assert A.dtype == 'float'
    assert b.shape == (1, 1)
    assert len(G) == 2 * len(X)
    assert len(h) == 2 * len(X)

    return solve_quadratic_program(P, q, A, b, G, h)

def solve_quadratic_program(P, q, A, b, G, h):
    '''Uses cvxopt to solve the quadratic program.'''
    P, q, A, b, G, h = [cvxopt.matrix(var) for var in [P, q, A, b, G, h]]
    minimization = cvxopt.solvers.qp(P, q, G, h, A, b)
    lagr_mult = np.ravel(minimization['x'])
    return lagr_mult


def extract_parameters(X, t, kernel, lagr_mult, threshold=1e-7):
    '''Computes the intercept from the support vector constraints.
    
    Inputs
        X:         predictors
        t:         targets
        kernel:    a kernel to be used
        lagr_mult: the Lagrange multipliers obtained by solving the dual QP
        threshold: threshold for choosing support vectors
    
    Returns
        lagr_mult: lagrange multipliers for the support vectors
        svs:       set of support vectors
        sv_labels: targets t_n for the support vectors
        intercept: computed intercept (also called bias)
    '''
    t = np.array([-1. if l == 0 else 1. for l in t])
    
    # ---------------- INSERT CODE ----------------------
    N=len(t)
    intercept=np.array(t)
    
    for i in range(N):
        s=0
        for j in range(N):
            s+=t[j]*lagr_mult[j]*X[j]@X.T
        intercept-=s
    
    intercept=np.mean(intercept)
    svs=np.zeros(2)
    sv_labels=np.zeros(N)
     
    for i in range(N):
        svs+=lagr_mult[i]*t[i]*X[i]
        sv_labels+=intercept+kernel(X[i],X.T)*lagr_mult*t
    # ---------------- END CODE -------------------------

    return lagr_mult, svs, sv_labels, intercept

In [14]:
# Training
lagr_mult=fit_svm(X_train,t_train,linear_kernel)
# Fit SVM on training data
extract_parameters(X_train, t_train, linear_kernel, lagr_mult,)

# Extract parameters


(1, 175)
     pcost       dcost       gap    pres   dres
 0: -3.3599e+02 -2.7495e-05  9e+02  3e+00  1e+00
 1: -1.0725e+02 -1.6784e+02  7e+01  3e-02  1e-02
 2: -1.3984e+02 -1.4602e+02  7e+00  3e-03  9e-04
 3: -1.3999e+02 -1.4006e+02  7e-02  3e-05  1e-05
 4: -1.4000e+02 -1.4000e+02  7e-04  3e-07  1e-07
 5: -1.4000e+02 -1.4000e+02  7e-06  3e-09  1e-09
Optimal solution found.


(array([1.        , 1.        , 0.66750816, 0.66811915, 0.66722297,
        1.        , 1.        , 0.6647988 , 1.        , 1.        ,
        0.67068015, 1.        , 0.66118482, 1.        , 1.        ,
        0.66495307, 0.66199543, 1.        , 0.66062643, 1.        ,
        1.        , 1.        , 0.66233809, 0.6641014 , 1.        ,
        0.66400874, 1.        , 0.99999999, 0.66094407, 0.66264546,
        0.66116519, 1.        , 0.66383289, 1.        , 0.67725611,
        0.66404356, 0.68073898, 0.65990356, 0.99999999, 1.        ,
        0.6639249 , 0.66280827, 0.67420636, 1.        , 0.66445642,
        1.        , 0.66335982, 0.66421049, 0.67569219, 0.6641338 ,
        0.66005008, 0.66383152, 0.66461809, 1.        , 0.66165353,
        0.68033624, 0.66413057, 1.        , 0.68101114, 0.67801923,
        1.        , 0.66796331, 0.67939716, 0.99999999, 0.66414117,
        0.6637172 , 0.66266996, 0.66356432, 1.        , 0.66077657,
        0.66437101, 0.66064454, 1.        , 0.66

### Task 1.2

Having learnt an SVM, we can use the calculated parameters to make predictions on novel samples.
- Implement a function `svm_predict(X, kernel, lagr_mult, svs, sv_labels, intercept)`.
- Use this function with the linear kernel and compute the test accuracy on the 2d dataset.
- Visualize the samples form the test set in a scatter plot colored by your predictions

In [ ]:
def svm_predict(X, kernel, lagr_mult, svs, sv_labels, intercept):
    ''' Given the learned parameters of the SVM, make a prediction on the test set.
    Inputs
        X:         predictors
        kernel:    a kernel to be used
        lagr_mult: the Lagrange multipliers obtained by solving the dual QP
        svs:       set of support vectors
        sv_labels: targets t_n for the support vectors
        intercept: computed intercept (also called bias)
    
    Returns
        prediction: predictions on novel samples
    '''  
    
    # ---------------- INSERT CODE ----------------------



    # ---------------- END CODE -------------------------
    
    return prediction

In [ ]:
# Testing
# make predictions for test set


In [ ]:
# visualize predictions


### Task 1.3

- Instead of using the linear kernel, use the Gaussian RBF kernel defined in Practical 3.
- Compare results on with both kernels with sklearn implementation (SVC)
- Visualize the predictions on the test set, the learned support vectors and the decision boundary for both kernels (Hint: Adapt the decision boundary plot from Practical 3).

In [ ]:
def rbf_kernel(a, b):
    # ---------------- INSERT CODE ----------------------



    # ---------------- END CODE -------------------------

In [ ]:
# Fit SVM with rbf kernel and calculate the test accuracy


In [ ]:
# Fit SVM using sklearn and calculate the test accuracy


In [ ]:
# Visualize


# Task 2: Decision Trees

Next, we will implement a simple decision tree classifier using the Wine dataset, one of the standard sklearn datasets. 

We will use the Gini impurity as a criterion for splitting. It is defined for a set of labels as
$$ G = \sum_{i=0}^C p(i) * (1- p(i)) $$

Given labels $l$ and split $l_a$ and $l_b$, the weighted removed impurity can be computed by $G(l) - \frac{|l_a|}{|l|}G(l_a) - \frac{|l_b|}{|l|}G(l_b)$.

Here is a simple explanation of the Gini impurity that you may find useful: https://victorzhou.com/blog/gini-impurity/


### Task 2.1

1. Plot the distribution of the first feature of for each class of the wine dataset.
2. Implement a function `gini_impurity(t)` that computes the Gini impurity for an array of labels `t`.
3. Calculate the removed Gini impurity for a split after 50 samples, i.e. between `t[:50]` and `t[50:]`.

In [ ]:
# Load Wine dataset and split into train+test set

X, t = load_wine(return_X_y=True)
X_train, X_test, t_train, t_test = split_data(X, t)

In [ ]:
# Plot distribution


In [ ]:
# Compute Gini impurity


### Task 2.2
For each of the first 12 features, compute the remove Gini impurity for every possible split. Visualize the removed Gini impurity per feature across all splits. Which is the optimal split?

In [ ]:
# Plotting


### Task 2.3

1. Implement a function `build_tree(X, t, depth)` which recursively builds a tree. Use the classes `Node` and `Leaf` as a data structure to build your tree.
2. Implement a function `predict_tree(tree, x)` which makes a prediction for sample `x`. Obtain scores for the `wine` dataset and compare to `sklearn.tree.DecisionTree`.
3. Switch back to the synthetic 2d dataset from the beginning (kernel methods). Compute scores and visualize the decisions in a 2d grid.

In [ ]:
class Node:
    def __init__(self, left, right, n_feat, threshold):
        self.left = left
        self.right = right
        self.n_feat = n_feat
        self.threshold = threshold


class Leaf:
    def __init__(self, label):
        self.label = label


In [ ]:
# Implement recursive tree function

def build_tree(X, t, depth, max_depth=3, n_labels=2):
    
    # ---------------- INSERT CODE ----------------------



    # ---------------- END CODE -------------------------

    
def predict_tree(node, x):
    
    # ---------------- INSERT CODE ----------------------



    # ---------------- END CODE -------------------------

In [ ]:
# Build tree

tree = build_tree(X_train, t_train, 0, max_depth=3, n_labels=3)

In [ ]:
# Calculate training and test scores


In [ ]:
# Calculate test score using sklearn


In [ ]:
# Calculate test score for synthetic 2D dataset


In [ ]:
# Visualize
